In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
tf.executing_eagerly()

True

In [ ]:
class GAN(keras.Model):
    def __init__( self , noise_vector_dim):
        # 繼承Model的init， 第一個參數填這個class的名字
        super(GAN, self).__init__()
        # discriminator: 輸入圖片
        self.noise_vector_dim = noise_vector_dim
        self.discriminator = tf.keras.Sequential([ # 圖片 => 是否為generator製造
          tf.keras.layers.Conv2D(2, 3, activation='relu', input_shape=(28,28,1)), #input "一筆" data的大小 (width ,height,deepth)  
          tf.keras.layers.Flatten(),                    
          tf.keras.layers.Dense(1 , activation="softmax"),
          ])
        self.generator = tf.keras.Sequential([  # vector => 圖片
          tf.keras.layers.Input(shape=(noise_vector_dim)),                 # 輸入一個noise vector                                     
          tf.keras.layers.Dense(7 * 7 * 64, activation="relu"),     # 經過以下變換回傳一張圖片
          tf.keras.layers.Reshape((7,7,64)),
          tf.keras.layers.Conv2DTranspose(64, kernel_size=3, strides=2, activation='relu', padding='same'),
          tf.keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'),
          tf.keras.layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')
          ])
        # compile
        self.discriminator.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy()  )
        # self.generator.compile(optimizer="adam",loss=tf.keras.losses.MeanSquaredError()  )  實驗1:單純讓G學的input越像越好 , 有可能G早早卡在瓶頸，D早早太強， 如果mini batch慢慢餵，最後G也會卡住生出一坨一樣的東西(類似平均取最小值)  
        self.generator.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy()  )

        # *****關鍵***** 幫助G成長， G生成 => D辨認 => 結果如果被認為是0，就是被抓到，要有loss (==>視為每張圖G都認為自己是真的(1),被抓出來表示錯誤了(0))
        self.stacked_generator_and_discriminator = self._stacked_generator_and_discriminator()
        self.stacked_generator_and_discriminator.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy()  )
    

    def _stacked_generator_and_discriminator(self):
        self.discriminator.trainable = False #這一步不更新D

        model = tf.keras.Sequential()
        model.add(self.generator)
        model.add(self.discriminator)

        return model

    def train_one_step(self ,data_x ):

        # 先後不可顛倒!因為Generator 是要依Discriminator去調整! (而不是和input像就好，參考實驗1)
        # fake picture
        fake = self.generator.predict(np.random.rand(len(x_train),self.noise_vector_dim))
        real = data_x.reshape(data_x.shape[0],data_x.shape[1],data_x.shape[2],1) 

        #sample data for discriminator 
        length_of_data_for_discriminator = 1000  #實驗2: (可以調高試試)一次訓練資料太多可能會有一方輾壓，造成訓練失敗
        real_index = np.random.choice(len(real) , length_of_data_for_discriminator//2)
        fake_index = np.random.choice(len(fake) , length_of_data_for_discriminator//2)

        x = np.concatenate ( (real[real_index] , fake[fake_index]) , axis = 0)  #choice 只能選一維，可以先選index ,再選位置!!! ，注意concatenate( (a,b) , axis)
        y = np.concatenate ( (np.array([1 for i in range(length_of_data_for_discriminator//2)]) , np.array([0 for i in range(length_of_data_for_discriminator//2)])),axis = 0 )
        x,y = shuffle(x,y) #sklearn
        # training discriminator
        print("discriminator training")
        self.discriminator.fit(x,y)

        # Generator 一樣先生出資料 ==> 給discriminator辨識 ==> 用結果去更新G (可以把標籤顛倒，理由上面說過 )

        #sample data for generator
        length_of_data_for_generator = 1000 

        noise = np.random.rand(length_of_data_for_generator,self.noise_vector_dim)
        all_true_label = np.array([1 for i in range(length_of_data_for_generator)])  #看自己和"全部都騙過"相差多遠


        #training generator
        print("generator training")
        self.stacked_generator_and_discriminator.fit(noise , all_true_label)


    
    def train(self , data_x , epochs=5):
        for i in range(epochs):
            print(i+1,"th step")
            self.train_one_step(data_x)
            # random show image 
            one_pic = self.generator.predict(np.random.rand(1,self.noise_vector_dim))
            plt.matshow(one_pic[0].reshape(28,28), cmap=plt.get_cmap('gray'))

In [ ]:
gan = GAN(10)

gan.train(x_train ,epochs=100)

In [ ]:
# 隨機print出圖片
test = np.random.rand(10,10)
print(test)
pics= gan.generator.predict(test)

for pic in pics:
    plt.matshow(pic.reshape(28,28), cmap=plt.get_cmap('gray'))

In [ ]:
# 訓練到一半D把G打趴，可能會出現全黑的畫面 (G放棄了QQ)